In [1]:
!pip install ipython-autotime --quiet
%load_ext autotime

time: 0 ns (started: 2022-04-04 18:43:05 +02:00)


In [2]:
import torch
import pandas as pd
import random
import pickle

time: 2.27 s (started: 2022-04-04 18:43:06 +02:00)


In [3]:
import sys
import os

try:
    from google.colab import drive
    drive.mount('/content/drive/')
except:
    # local exec
    logDir = os.path.join(os.getcwd(), 'logs/')
    saveDir = os.path.join(os.getcwd(), 'model/')

    genIngList = os.path.join(os.getcwd(), 'generatedLists.pkl')


    sys.path.append(os.getcwd())
else: 
    rootDir = '/content/drive/MyDrive/'

    TIMESTAMP = '2022_03_30'

    logDir = rootDir + 'Colab Notebooks/recime/LSTM/torch/logs/'
    saveDir = rootDir + 'Colab Notebooks/recime/LSTM/torch/model/'

    genIngList = rootDir + 'TP2/Network/VAE/generatedLists.pkl'


    sys.path.append(rootDir + 'Colab Notebooks/recime/LSTM/torch')


if not os.path.exists(logDir):
  print('ERROR: No run logs available')
if not os.path.exists(saveDir):
  print('ERROR: No saved model available')

time: 15 ms (started: 2022-04-04 18:43:08 +02:00)


In [4]:
import src.instGen as instGen
import src.titleGen as titleGen

time: 8.52 s (started: 2022-04-04 18:43:08 +02:00)


In [5]:
hpTitle = titleGen.HyperParams()
hpTitle.setHyperParams(saveDir + 'title/hp.yml')

hpInst = instGen.HyperParams()
hpInst.setHyperParams(saveDir + 'instruction/hp.yml')

device='cpu'

with open(saveDir + 'title/titleTok.pkl', 'rb') as inp:
    titleTok = pickle.load(inp)

with open(saveDir + 'instruction/instTok.pkl', 'rb') as inp:
    instTok = pickle.load(inp)

time: 62 ms (started: 2022-04-04 18:43:17 +02:00)


In [6]:
titleModel = titleGen.EmbedLSTM(hpTitle, titleTok, device)
titleModel.load_state_dict(torch.load(saveDir + 'title/titleGenerator_model_state_dict.pt', map_location=torch.device(device)))

instModel = instGen.EmbedLSTM(hpInst, instTok, device)
instModel.load_state_dict(torch.load(saveDir + 'instruction/instGenerator_model_state_dict.pt', map_location=torch.device(device)))

<All keys matched successfully>

time: 141 ms (started: 2022-04-04 18:43:18 +02:00)


In [7]:
genIng = pd.read_pickle(genIngList)

time: 31 ms (started: 2022-04-04 18:43:18 +02:00)


In [8]:
for idx in range(len(genIng)):
  # inputIngs = genIng.loc[idx]['ingredient'].tolist()
  inputIngs = random.sample(genIng.loc[idx]['ingredient'].tolist(), len(genIng.loc[idx]['ingredient'].tolist()))
  amounts = genIng.loc[idx]['amount'].tolist()
  units = genIng.loc[idx]['unit'].tolist()

  titleSeq = titleGen.testInputPrep(inputIngs, titleTok)
  titleSample = titleGen.sample(titleModel, titleTok, 5, device, initial=titleSeq)

  instSeq  = instGen.testInputPrep(inputIngs + titleSample.split(), instTok)
  instSample = instGen.sample(instModel, instTok, 200, device, initial=instSeq).split('\n')

  print('Amount: {} \nUnit: {} \nIngredients: {} \nTitle: {} \nInstructions: {}\n'.format(amounts, units, inputIngs, titleSample, instSample))

Amount: [0.35911798, 0.2554059, 0.18835068, 0.25898218] 
Unit: ['teaspoon', 'centimetre', 'centimetre', 'teaspoon'] 
Ingredients: ['chicken', 'apples', 'salt', 'salt'] 
Title: cranberries cranberries cranberries cranberries cranberries 
Instructions: ['individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual